# Setup Environment

In [ ]:
#install EOS test framework
! python3 -m pip install ipyeos

#install Go Smart Contracts Development Kit
! python3 -m pip install gscdk

#install Python Toolkit for EOS
! python3 -m pip install pyeoskit

# install go
! wget https://go.dev/dl/go1.18.3.linux-amd64.tar.gz
    
! tar -C /usr/local -xzf go1.18.3.linux-amd64.tar.gz

# install binaryen
! wget https://github.com/WebAssembly/binaryen/releases/download/version_109/binaryen-version_109-x86_64-linux.tar.gz
! tar -C /usr/local -xzf binaryen-version_109-x86_64-linux.tar.gz

# Add go and binaryen to path
import os
os.environ['PATH'] += ":/usr/local/go/bin:/usr/local/binaryen-version_109/bin"

In [ ]:
# Initialize a new project with name hello
! eosio-go init hello
os.chdir('hello')

In [ ]:
!ls -l

In [ ]:
!cat main.go

# Building

In [ ]:
!./build.sh

# Testing

In [ ]:
!./test.sh